## Self-attention Generative Adversarial Capsulet Networks

#### Importing Modules

In [3]:
import os, sys, glob
import numpy as np
import zipfile
import skimage
from matplotlib import pyplot as plt
from PIL import Image
import tensorflow as tf
from keras import layers, models, optimizers
from keras import callbacks, backend
from keras.utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from utils import load_image, load_faces
from discriminator import discriminator_func
from generator import generator_func
from attention import self_attention

#### Dataset Preparation

In [ ]:
!wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip

with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

root = 'data_faces/img_align_celeba'
img_list = os.listdir(root)
print("Total Number of Images : ", len(img_list))
required_size = (64, 64)



In [ ]:
number_of_images = 50000
 

root = 'data_faces/img_align_celeba/'
dataset = load_faces(directory, number_of_images)
print('Loaded: ', dataset.shape)

In [ ]:
def generate_real_samples(dataset, n_samples):
	ix = np.random.randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = np.ones((n_samples, 1))
	return X, y
 

def generate_latent_points(latent_dim, n_samples):
	x_input = np.random.randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 

def generate_fake_samples(generator, latent_dim, n_samples):
	x_input = generate_latent_points(latent_dim, n_samples)
	X = generator.predict(x_input)
	y = np.zeros((n_samples, 1))
	return X, y

#### Building Discriminator

In [4]:
input_shape = (128, 128, 3)

discriminator = discriminator_func(input_shape)

print('DISCRIMINATOR:')

discriminator.summary()

discriminator.compile(loss = 'binary_crossentropy', optimizer = Adam(0.0004, 0.48), metrics = ['accuracy'])

DISCRIMINATOR:
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 124, 124, 16) 1216        input_2[0][0]                    
__________________________________________________________________________________________________
self_attention_3 (self_attentio (None, 124, 124, 16) 321         conv1[0][0]                      
__________________________________________________________________________________________________
leaky_re_lu_6 (LeakyReLU)       (None, 124, 124, 16) 0           self_attention_3[0][0]           
_____________________________________________________________________________

In [ ]:
#### 